In [22]:
import yfinance as yf
import numpy as np
import tensorflow as tf

class Brain:
    def __init__(self):
        self.model = tf.keras.Sequential()
        initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
        self.model.add(tf.keras.layers.Dense(80, activation='relu', input_shape=(75,), kernel_initializer=initializer))
        self.model.add(tf.keras.layers.Dense(10, activation='sigmoid', kernel_initializer=initializer))
        opt = tf.keras.optimizers.Adam()
       
        self.model.compile(loss='mse', optimizer=opt, metrics=['accuracy', 'mse'])

    def prediction(self, inputs):
        # Inputs is expected to be a matrix with rows as instances and columns as features
        # Ensure inputs are numpy array and properly shaped

        input_vector = np.array([inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5], inputs[6], inputs[7], inputs[8], inputs[9], inputs[10], inputs[11], inputs[12], inputs[13], inputs[14]])
        input_vector = input_vector.astype('float32')
        input_vector = np.reshape(input_vector, (1,75))
        
        pred = self.model.predict(input_vector)
        threshold = 0.5
        binary_outputs = (pred >= threshold).astype(int)
        # Example processing, adjust as needed:
        # Assuming you want the first three outputs in a specific format
        if binary_outputs.shape[1] >= 3:
            rtn = [binary_outputs[0][0], binary_outputs[0][1], binary_outputs[0][2]]
        else:
            rtn = [0, 0, 0]  # Default return if not enough outputs
        return rtn

    def set_weights(self, weights):
        self.model.set_weights(weights)
 
    def get_weights(self):
        return self.model.get_weights()

class Data:
    def __init__(self):
        self.data = yf.download('AAPL', period='5y', interval='1d')
        self.current_index = 0

    def get_current_price(self):
        if self.current_index < len(self.data):
            return self.data.iloc[self.current_index]['Close']
        else:
            return None

    def get_tensor(self):
        end_index = self.current_index + 1
        start_index = max(0, end_index - 15)
        return np.array(self.data.iloc[start_index:end_index])

    def next(self):
        if self.current_index < len(self.data) - 1:
            self.current_index += 1
        else:
            self.current_index = 0

class Agent:
    id = 0
    data = Data()

    def __init__(self):
        self.id = Agent.id
        Agent.id += 1
        self.input_dim = 75
        self.current_price = Agent.data.get_current_price()
        self.sliding_window = []
        for _ in range(14):
            self.sliding_window.append(np.array([0,0,0,0,0], dtype='float32'))
        self.sliding_window.append(Agent.data.get_tensor()[0][:5])
        self.brain = Brain()
        self.predictions = self.brain.prediction(self.sliding_window)
        self.active_trades = []
        self.num_trades = 0
        self.fitness = 0

        if self.predictions[0] > 0.5:
            self.action = 'hold'
        elif self.predictions[1] > 0.5:
            self.action = 'buy'
        else:
            self.action = 'sell'

        if self.action == 'hold':
            self.target = None
        elif self.action == 'sell':
            self.target = self.current_price * (1 - self.predictions[2])
        else:
            self.target = self.current_price * (1 + self.predictions[2])

    def __lt__(self, other):
        return self.fitness < other.fitness

    def output_from_input(self, input):
        if input.shape[0] == self.input_dim:
            self.predictions = self.brain.predict(input)
            return True
        return False

    def process_predictions(self):
        self.active_trades = []
        if self.predictions[0] > 0.5:
            self.action = 'hold'
        elif self.predictions[1] > 0.5:
            self.action = 'buy'
        else:
            self.action = 'sell'

        if self.action == 'hold':
            self.target = None
        elif self.action == 'sell':
            self.target = self.current_price * (1 - self.predictions[2])
        else:
            self.target = self.current_price * (1 + self.predictions[2])

    def make_trade(self):
        if self.action == 'buy':
            self.buy()
        elif self.action == 'sell':
            self.sell()

    def buy(self):
        if self.target > self.current_price * 1.10 or self.target < self.current_price:
            self.fitness -= 10
        else:
            stop_loss = 2 * self.current_price - self.target
            self.active_trades.append([self.current_price, self.target, stop_loss, 1])
            self.num_trades += 1

    def sell(self):
        if self.target < self.current_price * 0.90 or self.target > self.current_price:
            self.fitness -= 10
        else:
            stop_loss = 2 * self.current_price - self.target
            self.active_trades.append([self.current_price, self.target, stop_loss, 0])
            self.num_trades += 1

    def close_trades(self):
        remaining_trades = []
        for trade in self.active_trades:
            open_price, target, stop_loss, trade_type = trade
            if self.current_price >= target or self.current_price <= stop_loss:
                is_gain = self.current_price >= target if trade_type == 1 else self.current_price <= stop_loss
                returns = Agent.calculate_returns(open_price, self.current_price, is_gain)
                self.fitness += returns
            else:
                remaining_trades.append(trade)
        self.active_trades = remaining_trades

    @staticmethod
    def calculate_returns(open_price, close_price, is_gain):
        result = abs(close_price - open_price)
        return result if is_gain else -result

    def update_fitness(self):
        total_profit = 0
        for trade in self.active_trades:
            if trade[3] == 1:
                total_profit += self.current_price - trade[0]
        self.fitness += total_profit

    def one_cycle(self):
        if self.output_from_input(self.data.get_tensor()[0][:5]):
            self.process_predictions()
            self.make_trade()
        self.close_trades()
        self.update_fitness()
        self.data.next()
        self.sliding_window.append(Agent.data.get_tensor()[0][:5])
        self.sliding_window.pop()

class Gen:
    def __init__(self, population_size=100, generations=10, mutation_rate=0.01, crossover_rate=0.7):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = [Agent() for _ in range(population_size)]

    def evaluate_fitness(self):
        for agent in self.population:
            agent.one_cycle()  # Assumes one_cycle updates the agent's fitness

    def select_parents(self):
        # Tournament selection
        parents = []
        tournament_size = 5
        for _ in range(int(self.crossover_rate * self.population_size)):
            contenders = np.random.choice(self.population, tournament_size, replace=False)
            best = max(contenders, key=lambda x: x.fitness)
            parents.append(best)
        return parents

    def crossover(self, parent1, parent2):
        child = Agent()
        p1_weights = parent1.brain.get_weights()
        p2_weights = parent2.brain.get_weights()
        new_weights = []
        for p1_layer, p2_layer in zip(p1_weights, p2_weights):
            gene_cutoff = np.random.randint(0, p1_layer.size)
            new_gene = np.concatenate([p1_layer[:, :gene_cutoff], p2_layer[:, gene_cutoff:]], axis=1)
            new_weights.append(new_gene)
        child.brain.set_weights(new_weights)
        return child

    def mutate(self, agent):
        weights = agent.brain.get_weights()
        mutated_weights = []
        for weight in weights:
            if np.random.rand() < self.mutation_rate:
                mutation_matrix = np.random.uniform(-0.1, 0.1, weight.shape)
                weight += mutation_matrix
            mutated_weights.append(weight)
        agent.brain.set_weights(mutated_weights)

    def run_generation(self):
        self.evaluate_fitness()
        parents = self.select_parents()
        new_population = []

        while len(new_population) < self.population_size:
            parent1, parent2 = np.random.choice(parents, 2, replace=False)
            child1 = self.crossover(parent1, parent2)
            child2 = self.crossover(parent1, parent2)
            self.mutate(child1)
            self.mutate(child2)
            new_population.extend([child1, child2])

        self.population = new_population[:self.population_size]  # Ensure population size remains constant

    def simulate(self):
        for _ in range(self.generations):
            self.run_generation()
            best_fitness = max(agent.fitness for agent in self.population)
            print(f'Best fitness in current generation: {best_fitness}')

# Example of how to use the Gen class
gen_algo = Gen(population_size=10, generations=50, mutation_rate=0.05)
gen_algo.simulate()

# data = Data()
# print(data.get_tensor())

[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 32ms/step
6000
80


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed